# Abstract

This is a clone of the script at https://www.kaggle.com/ceshine/lgbm-starter which is intended to give an idea of how to structure the data for trainig

# Prelude 

## Configuration

In [1]:
DataSetPath = "/home/bryanfeeney/Workspace/OttomanDiviner/favorita/"

StoresPath   = DataSetPath + "stores.csv.gz"
ItemsPath    = DataSetPath + "items.csv.gz"
OilPricePath = DataSetPath + "oil.csv.gz"
HolidaysPath = DataSetPath + "holidays_events.csv.gz"
Transactions = DataSetPath + "transactions.csv.gz"
TrainData    = DataSetPath + "train-2017.csv.gz"
TestData     = DataSetPath + "test.csv.gz"
# TrainData    = DataSetPath + "train-2018.csv.gz"
# TestData     = DataSetPath + "query-2018.csv"

FutureDaysToCalculate=16
WeeksOfHistoryForFeature=8
WeeksOfHistoryForFeatureOnValidation=3

## Imports

In [2]:
from datetime import date, datetime, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# Intro to the Data

In [3]:
cumul_sales = pd.read_csv(
    TrainData, 
    usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    compression='gzip'
)


In [4]:

cumul_sales_query = pd.read_csv(
    TestData,
    usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
)

In [5]:
query_start_date = str(cumul_sales_query.iloc[0,1]).split(" ")[0]

In [6]:
query_start_date

'2017-08-16'

In [7]:
cumul_sales_query = cumul_sales_query.set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [8]:
cumul_sales.shape, cumul_sales_query.shape

((23808260, 5), (3370464, 2))

In [9]:
cumul_sales.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2017-01-01,25,103665,2.079442,False
1,2017-01-01,25,105574,0.693147,False
2,2017-01-01,25,105857,1.609438,False
3,2017-01-01,25,106716,1.098612,False
4,2017-01-01,25,108698,1.098612,False


<font color="red">DEBUG</font>

In [10]:
cumul_sales_query.head()

id  onpromotion
store_nbr item_nbr date                              
1         96995    2017-08-16  125497040        False
          99197    2017-08-16  125497041        False
          103501   2017-08-16  125497042        False
          103520   2017-08-16  125497043        False
          103665   2017-08-16  125497044        False

In [11]:
promo_variables_test = cumul_sales_query[["onpromotion"]].unstack(level=-1).fillna(False)

In [12]:
promo_variables_test.head()

onpromotion                                              \
date                2017-08-16 2017-08-17 2017-08-18 2017-08-19 2017-08-20   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103501         False      False      False      False      False   
          103520         False      False      False      False      False   
          103665         False      False      False      False      False   

                                                                           \
date               2017-08-21 2017-08-22 2017-08-23 2017-08-24 2017-08-25   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   

                                                                           \
date               2017-08-26 2017-08-27 2017-08-28 2017-08-29 2017-08-30   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   

                               
date               2017-08-31  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103501        False  
          103520        False  
          103665        False

<font color=red>DEBUG</font>

In [13]:
cumul_sales

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2017-01-01,25,103665,2.079442,False
1,2017-01-01,25,105574,0.693147,False
2,2017-01-01,25,105857,1.609438,False
3,2017-01-01,25,106716,1.098612,False
4,2017-01-01,25,108698,1.098612,False
5,2017-01-01,25,108786,0.693147,False
6,2017-01-01,25,108797,0.693147,False
7,2017-01-01,25,108862,0.693147,False
8,2017-01-01,25,108952,1.098612,False
9,2017-01-01,25,114790,1.098612,False


In [14]:
cumul_sales_query.iloc[0,:]

id             125497040
onpromotion        False
Name: (1, 96995, 2017-08-16 00:00:00), dtype: object

In [15]:
items = pd.read_csv(
    ItemsPath,
).set_index("item_nbr")

stores = pd.read_csv(
    StoresPath
).set_index("store_nbr")

In [16]:
cumul_sales_query

id  onpromotion
store_nbr item_nbr date                              
1         96995    2017-08-16  125497040        False
          99197    2017-08-16  125497041        False
          103501   2017-08-16  125497042        False
          103520   2017-08-16  125497043        False
          103665   2017-08-16  125497044        False
          105574   2017-08-16  125497045        False
          105575   2017-08-16  125497046        False
          105576   2017-08-16  125497047        False
          105577   2017-08-16  125497048        False
          105693   2017-08-16  125497049        False
          105737   2017-08-16  125497050        False
          105857   2017-08-16  125497051        False
          106716   2017-08-16  125497052        False
          108079   2017-08-16  125497053        False
          108634   2017-08-16  125497054        False
          108696   2017-08-16  125497055        False
          108698   2017-08-16  125497056        False
          108701   2017-08-16  125497057         True
          108786   2017-08-16  125497058        False
          108797   2017-08-16  125497059         True
          108831   2017-08-16  125497060        False
          108833   2017-08-16  125497061        False
          108862   2017-08-16  125497062        False
          108952   2017-08-16  125497063        False
          111223   2017-08-16  125497064        False
          111397   2017-08-16  125497065        False
          112830   2017-08-16  125497066        False
          114778   2017-08-16  125497067        False
          114790   2017-08-16  125497068         True
          114799   2017-08-16  125497069        False
...                                  ...          ...
54        2127921  2017-08-31  128867474        False
          2127992  2017-08-31  128867475        False
          2128628  2017-08-31  128867476        False
          2128799  2017-08-31  128867477        False
          2129334  2017-08-31  128867478        False
          2129350  2017-08-31  128867479        False
          2129387  2017-08-31  128867480        False
          2129515  2017-08-31  128867481        False
          2129616  2017-08-31  128867482        False
          2129678  2017-08-31  128867483        False
          2129786  2017-08-31  128867484        False
          2129790  2017-08-31  128867485        False
          2129892  2017-08-31  128867486        False
          2129994  2017-08-31  128867487        False
          2130131  2017-08-31  128867488        False
          2130219  2017-08-31  128867489        False
          2130265  2017-08-31  128867490        False
          2130352  2017-08-31  128867491        False
          2130474  2017-08-31  128867492        False
          2130521  2017-08-31  128867493        False
          2130526  2017-08-31  128867494        False
          2130553  2017-08-31  128867495        False
          2131010  2017-08-31  128867496        False
          2131572  2017-08-31  128867497        False
          2131699  2017-08-31  128867498        False
          2132163  2017-08-31  128867499        False
          2132318  2017-08-31  128867500        False
          2132945  2017-08-31  128867501        False
          2132957  2017-08-31  128867502        False
          2134244  2017-08-31  128867503        False

[3370464 rows x 2 columns]

In [17]:
cumul_sales.shape

(23808260, 5)

In [18]:
cumul_sales_query.shape

(3370464, 2)

In [19]:
items.shape

(4100, 3)

## Select only Last Three Months

This is a peculiar one, and it **games the benchmark** in a not great way. Essentially it uses the last 11 weeks of data before the prediction threshold to predict what's happening next

In [20]:
nowtime = datetime.now()
now = date(nowtime.year, nowtime.month, nowtime.day)

# How far back to go to start generating trend features for demand
data_start             = now - timedelta(7*11) + timedelta(1)
training_history_start = now - timedelta(7*WeeksOfHistoryForFeature) + timedelta(1)
validation_start       = now - timedelta(7*WeeksOfHistoryForFeatureOnValidation) + timedelta(1)



In [21]:
data_start, training_history_start, query_start_date

(datetime.date(2018, 3, 14), datetime.date(2018, 4, 4), '2017-08-16')

In [22]:
cumul_sales = cumul_sales[cumul_sales.date.isin(
    pd.date_range(data_start, periods=7 * 11))].copy()


In [23]:
cumul_sales.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion


In [24]:
cumul_sales.shape

(0, 5)

In [25]:
cumul_sales.iloc[-1,:]

IndexError: single positional indexer is out-of-bounds

## Creating Promotion Variables

So this is a tricky. If one presumes that on-promotion will lead to a boost in demand, if if we presume we'll know *whats on promotion in advance*, then we can create variables to say that this product will be on promotion 1, 2, 3, ... 16 days from now (16 days in the future is the target)

In this case, this is also peculiar, there is a column for every single day!

In [ ]:
promo_variables = cumul_sales.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]]

In [ ]:
promo_variables.head()

In [ ]:
promo_variables = cumul_sales.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)



In [ ]:
promo_variables.head()

In [ ]:
promo_variables.columns = promo_variables.columns.get_level_values(1)

promo_variables_query = cumul_sales_query[["onpromotion"]].unstack(level=-1).fillna(False)
promo_variables_query.columns = promo_variables_query.columns.get_level_values(1)
promo_variables_query = promo_variables_query.reindex(promo_variables.index).fillna(False)

promo_variables_train_and_query = pd.concat([promo_variables, promo_variables_query], axis=1)


In [ ]:
promo_variables.shape, items.shape[0] * stores.shape[0]

In [ ]:
cumul_sales.shape, cumul_sales_query.shape

#  Unstack unit sales - do it across all days in a sliding window

Ah... they're creating a multi-task learning problem

In [ ]:
cumul_sales = cumul_sales.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
cumul_sales.columns = cumul_sales.columns.get_level_values(1)
cumul_sales.shape

In [ ]:
cumul_sales.head()

## Make items match other data frames

They're sacraficing generability

In [ ]:
items = items.reindex(cumul_sales.index.get_level_values(1))
items.head()

In [ ]:
items.shape

## Time futzing

In [ ]:
# Return that portion of the data frame that corresponds to the time period
#   beginning "minus" days before "dt" and extending for "periods" days
def get_timespan(df, dt, minus, periods):
    return df[
        pd.date_range(dt - timedelta(days=minus), periods=periods)
    ]

In [ ]:
def prepare_dataset(cumul_sales, promo_variables_train_and_query, start_date, is_train=True):
    X = pd.DataFrame({  # Mean target for different retrospective timespans & total # promotions
        "mean_3_2017": get_timespan(cumul_sales, start_date, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(cumul_sales, start_date, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(cumul_sales, start_date, 14, 14).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_variables_train_and_query, start_date, 14, 14).sum(axis=1).values
    })
    for i in range(16):  # Promotions on future days
        X["promo_{}".format(i)] = promo_variables_train_and_query[
            start_date + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = cumul_sales[  # Target values for future days
            pd.date_range(start_date, periods=16)
        ].values
        return X, y
    return X

In [ ]:
promo_variables_train_and_query.shape

In [ ]:
training_history_start, validation_start, now

In [ ]:
promo_variables

In [ ]:
print("Preparing dataset...")
X_l, y_l = [], []
for i in range(4):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(cumul_sales, promo_variables_train_and_query, training_history_start + delta)
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

X_validate, y_validate = prepare_dataset(cumul_sales, promo_variables_train_and_query, validation_start)

X_query = prepare_dataset(cumul_sales, promo_variables_train_and_query, now, is_train=False)

In [ ]:
X_train.shape, X_validate.shape, X_query.shape

This dataset is **super gamey**. They're using the means for the week, fortnight, and last three days, and then seeing how to permute it to generate values for the following window of time. It's hardcoded to product IDs, not categories.

It does however, permit multi-task learning, and therefore better representation learning

It does not incorporate any information about seasonality at all, and so would fall arse over face at Christmas



In [ ]:
print("Training and predicting models...")
params = {
    'num_leaves': 2**5 - 1,
    'objective': 'regression_l2',
    'max_depth': 8,
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 4
}

In [26]:
MAX_ROUNDS = 1000
validate_pred = []
query_pred = []
cate_vars = cat
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 4) * 0.25 + 1
    )
    
    dvalidate = lgb.Dataset(
        X_validate, label=y_validate[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dvalidate], early_stopping_rounds=50, verbose_eval=50
    )
    
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    
    
    validate_pred.append(bst.predict(
        X_validate, num_iteration=bst.best_iteration or MAX_ROUNDS))
    
    query_pred.append(bst.predict(
        X_query, num_iteration=bst.best_iteration or MAX_ROUNDS))

NameError: name 'categorical_feature' is not defined

In [ ]:
print("Validation mse:", np.sqrt(mean_squared_error(
    np.expm1(y_validate), np.expm1(np.array(validate_pred)).transpose())))

In [ ]:
validate_pred

In [ ]:
query_pred

In [ ]:
print("Making submission...")
y_query = np.array(query_pred).transpose()
df_preds = pd.DataFrame(
    y_query, index=cumul_sales.index,
    columns=pd.date_range(query_start_date, periods=16)
).stack().to_frame("unit_sales")
df_preds.to_csv("/tmp/preds-2018.csv")


In [ ]:
df_preds

In [ ]:
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

In [ ]:

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)

In [ ]:
submission

# Further Improvements

This is based on the work in this file: https://www.kaggle.com/vrtjso/lgbm-one-step-ahead

This was apparently in the top 10% at one point.

In [ ]:
df_train = pd.read_csv(
    TrainData, usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    TestData, usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    ItemsPath,
).set_index("item_nbr")

df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train

promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

items = items.reindex(df_2017.index.get_level_values(1))

def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values
    })
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

print("Preparing dataset...")
t2017 = date(2017, 5, 31)
X_l, y_l = [], []
for i in range(6):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

print("Training and predicting models...")
params = {
    'num_leaves': 31,
    'objective': 'regression',
    'min_data_in_leaf': 300,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 500
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=100
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb.csv', float_format='%.4f', index=None)


In [ ]:
print("Validation mse:", mean_squared_error(
    np.expm1(y_validate), np.expm1(np.array(validate_pred)).transpose()))

In [ ]:
np.sqrt(275), np.sqrt(247)